In [ ]:
!pip install llamaapi  meta-ai-api

In [ ]:
!pip install pycaret

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
import pandas as pd
from google.colab import drive
import json
import numpy as np

In [ ]:
drive.mount('/content/drive')

In [ ]:
path_main_data='/content/drive/MyDrive/diplom_data/diplom_data.csv'
path_f_comments='/content/drive/MyDrive/diplom_data/comments.csv'
path_group_info='/content/drive/MyDrive/diplom_data/groups_info.csv'
path_group_info1='/content/drive/MyDrive/diplom_data/groups_info1.csv'
path_user_info='/content/drive/MyDrive/diplom_data/users_info.csv'
path_user_info1=path_user_info='/content/drive/MyDrive/diplom_data/users_info.csv'
path_user_activity='/content/drive/MyDrive/diplom_data/user_activity.csv'
path_user_activity1='/content/drive/MyDrive/diplom_data/user_activity.csv'

In [ ]:
diplom_data=pd.read_csv(path_main_data)
group_info=pd.read_csv(path_group_info)
group_info1=pd.read_csv(path_group_info1)
user_info=pd.read_csv(path_user_info)
user_info1=pd.read_csv(path_user_info1)
user_activity=pd.read_csv(path_user_activity)
user_activity1=pd.read_csv(path_user_activity1)

In [ ]:
f_comments=pd.read_csv(path_f_comments)

In [ ]:
def make_name(text):
  if '/' in text:
      return text.split('/')[-1]
  return text


In [ ]:
def cleae_surname(text):
  if len(text.split(' '))>=2:
    return ' '.join(text.split(' ')[1:])
  return text


In [ ]:
diplom_data["nics"]=diplom_data['Ссылка на профиль ВК'].apply(make_name)

In [ ]:
diplom_data["Ваше ФИО"]=diplom_data['Ваше ФИО'].apply(cleae_surname)

In [ ]:
diplom_data["nics"]=diplom_data["nics"].str.replace('@', '')
diplom_data

In [ ]:
group_info = group_info.drop(group_info[group_info['ID пользователя'] == 325067276].index)

In [ ]:
group_info=pd.concat([group_info, group_info1], axis=0,ignore_index=True)

In [ ]:
group_info = group_info.dropna()
group_info.reset_index(drop=True, inplace=True)
group_info

In [ ]:
group_info=group_info.groupby('ID пользователя').head(10)
group_info

In [ ]:
group_info.reset_index(drop=True, inplace=True)
group_info

In [ ]:
only_group_info= group_info.drop_duplicates(subset=['ID группы'], keep='first')
only_group_info

In [ ]:
only_group_info.reset_index(drop=True, inplace=True)

In [ ]:
vk_topics = [
    "Музыка",
    "Фильмы",
    "Игры",
    "Спорт",
    "Образование",
    "Наука",
    "Технологии",
    "IT",
    "Мода",
    "красота",
    "Здоровье",
    "медицина",
    "Юмор",
    "Новости",
    "политика",
    "Путешествия",
    "Кулинария",
    "Авто",
    "Дом",
    "Литература",
     "искусство",
    "Психология",
    "Бизнес",
    "отношения",
    "Аниме",
    "Манга",
]

In [ ]:
print(str(vk_topics))

In [ ]:
token="LL-dN2DynU8X8WYvmIKDuKUNZlJM8AypFD19NNxhfGCCOLWshQZ3NSOCuKvga3AhXeP"

In [ ]:
promt="определи какой из тем {topic} принадлежит сообщество из социальной сети по названию {name} и описанию {desc}.В ответе только одну тему одним словом из заданного списка"

In [ ]:
import time
def attempts(func):
    def wrapper(*args, **kwargs):
        res = None
        a = 0
        while res is None or a<5:
            a += 1
            try:
                return func(*args, **kwargs)
            except Exception as e:
                if a < 5:
                    print('!!!SLEEP!!!')
                    time.sleep(2)
                else:
                    print(f'EXCEPTION: {e}')
                    return None

    return wrapper

In [ ]:
llama = LlamaAPI(token)

In [ ]:
@attempts
def get_api(messege):
  resp= llama.run(messege)
  return resp

In [ ]:
result={}
def get_info(only_group_info):
  for i in range(14,len(only_group_info)):
    print(f"{only_group_info['Название группы'][i]}")

    api_request_json = {
      "messages": [
          {"role": "user", "content": promt.format(topic=vk_topics, name=only_group_info['Название группы'][i],desc=only_group_info['Описание'][i])}],
      }
    response = get_api(api_request_json)
    res=response.json().get('choices')[0].get("message", None).get("content")
    print(f"{i}:{res}")
    result[i]=res
  return result



In [ ]:
my_res=get_info(only_group_info)

In [ ]:
import random

In [ ]:
result=[random.choice(vk_topics) for _ in range(len(only_group_info))]

In [ ]:
only_group_info['theme']=result
only_group_info

In [ ]:
only_group_info=only_group_info.drop(columns=['ID пользователя','Название группы','Тип','Описание'])
only_group_info

In [ ]:
df_merged = pd.merge(group_info, only_group_info, on='ID группы', how='left')
df_merged

In [ ]:
t=df_merged.groupby('ID пользователя')['ID группы'].count()

In [ ]:
t=df_merged.groupby('ID пользователя')['theme'].apply(set).reset_index().apply(list)
t['theme']=t['theme'].apply(list)
t

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

In [ ]:
import torch
def get_embeddings(texts):
    # Токенизация
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

    # Получение эмбеддингов
    with torch.no_grad():
        outputs = model(**inputs)

    # Усреднение по всем токенам для получения одного эмбеддинга на предложение
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

In [ ]:
t['embeddings'] = t['theme'].apply(lambda topics: get_embeddings(topics).numpy())

In [ ]:
t.columns

In [ ]:
result_vec=df_merged.pivot_table(index='ID пользователя', columns='theme', aggfunc='size', fill_value=0)

In [ ]:
result_vec

In [ ]:
result_vec=result_vec.applymap(lambda x: 1 if x > 0 else 0)
result_vec

In [ ]:
diplom_data=diplom_data.rename(columns={'id':'ID пользователя'})

In [ ]:
diplom_data = pd.merge(diplom_data, t, on='ID пользователя', how='left')
diplom_data

In [ ]:
diplom_data = diplom_data.dropna()
diplom_data.reset_index(drop=True, inplace=True)
diplom_data

In [ ]:
diplom_data = diplom_data.astype({col: 'int' for col in diplom_data.columns if diplom_data[col].dtype == 'float'})
diplom_data

In [ ]:
def generate_random_list_with_repeats(original_list, min_repeats=10, max_repeats=25):
    random_list = []
    for item in original_list:
        repeats = random.randint(min_repeats, max_repeats)
        random_list.extend([item] * repeats)
    random.shuffle(random_list)
    return random_list

In [ ]:
res_comment=generate_random_list_with_repeats(diplom_data['ID пользователя'].tolist())

In [ ]:
df_comments=pd.DataFrame({'id':res_comment,'comment':f_comments['clean_text'][:len(res_comment)].tolist()})
df_comments

In [ ]:
ts=Translator()

In [ ]:
def translate(text):
  return ts.translate(text, dest='ru').text

In [ ]:
df_comments['ru_comment']=df_comments.comment.apply(translate)

In [ ]:
df_comments=df_comments.drop(columns=['comment'])

In [ ]:
!pip install transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline


pipe = pipeline("text-classification", model="sismetanin/sbert-ru-sentiment-rusentiment")

In [ ]:
pipe('с уважением дмитрий')

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="sismetanin/rubert-ru-sentiment-rusentiment")

In [ ]:
def get_label(text):
  return pipe(text)[0]['label']

In [ ]:
df_comments['label']=df_comments.ru_comment.apply(get_label)

In [ ]:
df_comments.label.value_counts()

In [ ]:
category_counts = df_comments['label'].value_counts()
category_counts

In [ ]:
category_counts = df_comments.groupby(['id', 'label']).size().unstack(fill_value=0)
category_counts

In [ ]:
total_counts = category_counts.sum(axis=1)
total_counts

In [ ]:
category_b_percentage = category_counts['LABEL_0'] / total_counts
category_b_percentage

In [ ]:
diplom_data['toxic'] = diplom_data['ID пользователя'].map(lambda user_id: 1 if category_b_percentage.loc[user_id] > 0.17 else 0)
diplom_data

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
diplom_data['toxic'].value_counts().plot(kind='bar', color=['blue', 'orange'])

In [ ]:
grouped = diplom_data.groupby(['Ваша учебная группа', 'toxic']).size().unstack(fill_value=0)

# Переименовываем колонки для удобства
grouped.columns = ['Не токсичные', 'Токсичные']

# Шаг 3: Построение графика
grouped.plot(kind='bar', stacked=True, figsize=(10, 6))


In [ ]:
diplom_data['Ваша учебная группа'].astype(int).value_counts().index

In [ ]:
diplom_data['Ваша область учебных интересов'].value_counts().plot(kind='bar', color=['blue', 'orange'])

In [ ]:
group_interest=diplom_data.groupby('Ваша учебная группа').sum()

In [ ]:
group_interest

In [ ]:
grouped = diplom_data.groupby(['Ваша учебная группа', 'Ваша область учебных интересов']).size().unstack().fillna(0)
grouped.plot(kind='bar',  figsize=(12, 8))

In [ ]:
grouped_theme = diplom_data.groupby('Ваша учебная группа').sum()

In [ ]:
grouped_theme=grouped_theme.drop(columns=set(grouped_theme.columns)-set(vk_topics))
grouped_theme

In [ ]:
grouped_theme.plot(kind='bar',  figsize=(12, 8))
plt.legend(title='Интересы', bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
grouped = diplom_data.groupby(['Ваша учебная группа', 'toxic']).size().unstack(fill_value=0)

# Переименовываем колонки для удобства
grouped.columns = ['Не токсичные', 'Токсичные']

# Шаг 3: Построение графика
grouped.plot(kind='bar', stacked=True, figsize=(10, 6))


In [ ]:
plt.figure(figsize=(10, 6))
topic_distribution.plot(kind='bar')


In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(diplom_data['Ваш егэ бал по математике'], kde=True, bins=10)

In [ ]:
sns.histplot(diplom_data['Ваш ЕГЭ бал по физике/инфе'], kde=True, bins=10)

In [ ]:
sns.histplot(diplom_data['Ваш ЕГЭ бал по русскому языку'], kde=True, bins=10)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
diplom_data.columns

In [ ]:
diplom_data['Ваша область учебных интересов'] = diplom_data['Ваша область учебных интересов'].apply(get_embeddings)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['Ваша область учебных интересов','sex'])
    ])
encoded_features = preprocessor.fit_transform(diplom_data)

In [ ]:
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=list(preprocessor.get_feature_names_out()))
result_df = pd.concat([diplom_data, encoded_df], axis=1)

In [ ]:
result_df=result_df.drop(columns=['Отметка времени','Ваша учебная группа','Ссылка на профиль ВК','Ваша область учебных интересов','sex','nics'])
result_df

In [ ]:
result_df=result_df.astype({col: 'float' for col in result_df.columns if result_df[col].dtype == 'int'})
result_df=result_df.astype({'ID пользователя':'int'})
result_df

In [ ]:
result_df_clust=result_df.drop(columns=['ID пользователя','Ваше ФИО'])

In [ ]:
!pip install hdbscan umap-learn

In [ ]:
import hdbscan
import umap

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
result_df_clust_scal = scaler.fit_transform(result_df_clust)


In [ ]:
scaled_df = pd.DataFrame(result_df_clust_scal, columns=result_df_clust.columns)
scaled_df

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,min_samples=3, metric='chebyshev' , alpha=1.3)
result_df_clust['cluster'] = clusterer.fit_predict(scaled_df)
result_df_clust

In [ ]:
umap_reducer = umap.UMAP(n_neighbors=15, min_dist=0.1)
X_umap = umap_reducer.fit_transform(result_df_clust)

In [ ]:
X_umap

In [ ]:
sns.scatterplot( data=X_umap)

In [ ]:
from pycaret.clustering import *

In [ ]:
exp = ClusteringExperiment()

In [ ]:
exp.setup(data=result_df_clust, normalize=True)

In [ ]:
exp.models()

In [ ]:
custom_params = { 'metric':'manhattan' }

In [ ]:
kmeans_model = exp.create_model('birch', num_clusters=3)

In [ ]:
result_df_clust['cluster'] = exp.assign_model(kmeans_model)['Cluster']

In [ ]:
result_df_clust

In [ ]:
exp.plot_model(kmeans_model, plot='cluster')


In [ ]:
exp.plot_model(kmeans_model, plot='distribution')


In [ ]:
custom_params = {'threshold': 2.5, 'branching_factor': 80}

# Обучение модели BIRCH с дополнительными параметрами
birch_model = exp.create_model('birch',num_clusters=3, **custom_params)

In [ ]:
all_models = exp.models()
all_models

In [ ]:
result_df_clust['cluster_hdb'] = exp.assign_model(birch_model)['Cluster']

In [ ]:
result_df_clust

In [ ]:
exp.plot_model(birch_model, plot='distribution')

In [ ]:
exp.plot_model(birch_model, plot='cluster')

In [ ]:
diplom_data['result']=result_df_clust['cluster_hdb']

In [ ]:
diplom_data

In [ ]:
grouped_bf = diplom_data.groupby('Ваша учебная группа').apply(lambda x: x['Ваше ФИО'].tolist())
grouped_bf[102]

In [ ]:
grouped = diplom_data.groupby('result')[['Ваша учебная группа', 'result','Ваше ФИО']].apply(lambda x: x.values.tolist())
grouped['Cluster 0']

In [ ]:
grouped_theme = diplom_data.groupby('result').sum()

In [ ]:
grouped_theme=grouped_theme.drop(columns=set(grouped_theme.columns)-set(vk_topics))
grouped_theme

In [ ]:
grouped_theme.plot(kind='bar',  figsize=(12, 8))
plt.legend(title='Интересы', bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
grouped = diplom_data.groupby(['result', 'toxic']).size().unstack().fillna(0)
grouped.plot(kind='bar',  figsize=(12, 8))

In [ ]:
from sklearn.cluster import Birch
model = Birch(branching_factor=30, n_clusters=3, threshold=2.5)
model.fit(scaled_df)
pred = model.predict(scaled_df)
pred

In [ ]:
diplom_data['birch']=pred
diplom_data